Output dataset must have the following format:

[[‘Who is Shaka Khan?’, {‘entities’: [[7, 17, ‘PERSON’]]}],
 [‘I like London and Berlin.’,
  {‘entities’: [[7, 13, ‘LOC’], [18, 24, ‘LOC’]]}]]
  
  
#### Credits: 

https://www.youtube.com/watch?v=YLQvVpCXpbU&ab_channel=PradipNichite

https://spacy.io/usage/training#quickstart

In [1]:
import pandas as pd
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans

nlp = spacy.blank("en")
doc_bin = DocBin()

In [2]:
file_path = 'C:/Users/ernes/Git/dsa3101-2220-12-ds/Backend/Data/skills/lightcast_skills_queries-data_analysis_machine learning_ML_statistic.csv'
skills = pd.read_csv(file_path)

In [3]:
def find_start_index(skill, description):
    return [n for n in range(len(description)) if description.lower().find(skill.lower(), n) == n]

def find_end_index(skill, description):
    start_indices = find_start_index(skill, description)
    skill_length = len(skill)
    end_indices = [ind + skill_length for ind in start_indices]
    return end_indices

In [4]:
# Getting the first and last indexes of each skill in their skill description
skills['start_ind'] = skills.apply(lambda x: find_start_index(x.Skill, x.Skill_Description), axis=1)
skills['end_ind'] = skills.apply(lambda x: find_end_index(x.Skill, x.Skill_Description), axis=1)

In [5]:
skills_formatted_for_spacy = []
for ind, row in skills.iterrows():
    query, skill, description, start_ind_list, end_ind_list = row
    start_ind_list = start_ind_list
    end_ind_list = end_ind_list
    entities = []
    for i in range(len(start_ind_list)):
        start_ind = int(start_ind_list[i])
        end_ind = int(end_ind_list[i])
        entities.append([start_ind, end_ind, 'TECHNICAL_SKILL'])
    skills_formatted_for_spacy.append([description, {'entities': entities}])

In [6]:
for training_example in tqdm(skills_formatted_for_spacy): 
    text = training_example[0]
    labels = training_example[1]
    doc = nlp.make_doc(text)
    ents = []
    for k, entities in labels.items():
        for entity in entities:
            start, end, label = entity
            #print(start, end, label)
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print(f"Skipping entity: {str(doc)[start:end]}")
            else:
                #print(f"Added entity: {str(doc)[start:end]}")
                ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

100%|██████████████████████████████████████████████████████████████████████████████████| 487/487 [00:00<00:00, 1305.93it/s]

Skipping entity: Datagram
Skipping entity: Datagram
Skipping entity: datagram


In [7]:
# Run this line ONLY if you want to save it to your disk
# doc_bin.to_disk("train.spacy")